In [1]:
import datetime as dt

import pandas as pd

In [2]:
data = pd.read_csv("../data/train_data.csv")
data_ori = data.copy()
data.iloc[-1, 0] = "2019/11/24"
data.iloc[-1, :] = data.iloc[-1, :].fillna(0)

company_list = pd.read_csv("../data/company_list.csv")

In [3]:
data.tail()

,Date,VGSH,JEF,IVZ,KTCC,FBZ,PLOW,JBK,IAC,IVR,...,ASFI,JTA,MNR,TFSL,GTN,TYG,VIRC,BIS,WOOD,MASI
415,2019/10/27,60.417824,18.853819,16.638926,5.97,15.280874,48.024773,29.00,229.559998,13.800565,...,6.95,11.057277,14.815565,19.043219,16.830000,17.788372,4.01,16.251677,64.136101,147.509995
416,2019/11/3,60.288853,19.238792,17.527559,6.23,14.581332,51.991562,30.00,219.949997,14.149504,...,10.10,11.200258,14.385562,19.189333,19.980000,17.243061,4.09,16.082491,65.034508,142.949997
417,2019/11/10,60.397987,18.967628,17.163631,5.96,14.726692,52.336071,28.50,221.820007,13.913970,...,10.25,11.209790,14.543064,19.189333,19.049999,17.111437,3.93,15.754073,65.343956,149.770004
418,2019/11/17,60.388062,20.636860,17.045601,5.52,14.763033,52.089993,28.74,221.639999,14.044824,...,10.43,11.114470,14.463972,19.267258,19.850000,16.888838,3.68,14.709110,64.635216,153.350006
419,2019/11/24,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [4]:
data.iloc[:, 0] = pd.to_datetime(data.iloc[:, 0])

In [5]:
data_wide = data.copy()
data_wide.index = data_wide.iloc[:, 0]
data_wide = data_wide.drop("Date", axis=1)

In [6]:
from sklearn.preprocessing import LabelEncoder

data_columns = data_wide.columns

symbols = []
symbols += company_list.loc[:, "Symbol"].values.tolist()
symbols += data_columns.values.tolist()
symbols = list(set(symbols))

labelEncoder = LabelEncoder()
labelEncoder.fit(symbols)
data_wide.columns = labelEncoder.transform(data_columns)
company_list.loc[:, "Symbol"] = labelEncoder.transform(company_list.loc[:, "Symbol"])

In [7]:
lag1 = data_wide.shift(1)
lag2 = data_wide.shift(2)
lag3 = data_wide.shift(3)
lag4 = data_wide.shift(4)

In [8]:
ave4 = data_wide.shift(1).rolling(window=4).mean()
ave8 = data_wide.shift(1).rolling(window=8).mean()
ave10 = data_wide.shift(1).rolling(window=10).mean()
ave14 = data_wide.shift(1).rolling(window=14).mean()

#### あいであ
- industryごとにグルーピングして集計したものに対してラグ特徴量の作成する

In [9]:
# data_long = data_wide.stack()
data_long = data_wide.stack().reset_index()
data_long.columns = ["date", "symbol", "stockPrice"]
data_long.head()

,date,symbol,stockPrice
0,2011-11-13,6591,55.942184
1,2011-11-13,3468,17.649006
2,2011-11-13,3436,13.770864
3,2011-11-13,3656,4.770000
4,2011-11-13,2310,13.818835


In [10]:
def wide_to_long(df_wide, columns=["date", "symbol", "stockPrice"]):
    df_long = df_wide.stack().reset_index()
    df_long.columns = columns
    return df_long

In [11]:
feature_df_list = list(
    map(wide_to_long, [lag1, lag2, lag3, lag4, ave4, ave8, ave10, ave14])
)

In [12]:
data_long_merged = data_long.copy()
for feature_df in feature_df_list:
    data_long_merged = pd.merge(
        data_long_merged, feature_df, how="left", on=["date", "symbol"]
    )

/tmp/ipykernel_260/4059174339.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'stockPrice_x'} in the result is deprecated and will raise a MergeError in a future version.
  data_long_merged = pd.merge(


In [13]:
data_long_merged.columns = [
    "date",
    "symbol",
    "stockPrice",
    "stockPriceLag1",
    "stockPriceLag2",
    "stockPriceLag3",
    "stockPriceLag4",
    "stockPriceAve4",
    "stockPriceAve8",
    "stockPriceAve10",
    "stockPriceAve14",
]

data_long_merged_drop = data_long_merged.dropna(how="any")

In [68]:
# Industry(業界）のその日の平均株価

industry_list = (
    company_list.loc[:, ["Symbol", "Industry"]].dropna(how="any").reset_index(drop=True)
)
industry_list = industry_list[~industry_list.duplicated(subset='Symbol')]
tmp = data_long.loc[:, ["date", "symbol", "stockPrice"]].reset_index(drop=True)

stockPrice_group = pd.merge(
    tmp, industry_list, left_on="symbol", right_on="Symbol"
).groupby(["date", "Industry"])

stockPrice_by_industry_mean = (
    pd.DataFrame(stockPrice_group["stockPrice"].mean())
    .stack()
    .reset_index()
    .drop("level_2", axis=1)
    .set_index("date")
)
stockPrice_by_industry_min = (
    pd.DataFrame(stockPrice_group["stockPrice"].min())
    .stack()
    .reset_index()
    .drop("level_2", axis=1)
    .set_index("date")
)
stockPrice_by_industry_max = (
    pd.DataFrame(stockPrice_group["stockPrice"].max())
    .stack()
    .reset_index()
    .drop("level_2", axis=1)
    .set_index("date")
)
stockPrice_by_industry_median = (
    pd.DataFrame(stockPrice_group["stockPrice"].median())
    .stack()
    .reset_index()
    .drop("level_2", axis=1)
    .set_index("date")
)


def long_to_wide(df_input, column="Industry", value=0):
    df_wide = df_input.pivot(index=None, columns=column, values=value)
    return df_wide


def create_lag_features(df_input, column_join_word):
    shifts = range(1, 5)
    lags = [df_input.shift(s) for s in shifts]
    lags = [
        wide_to_long(
            item, columns=["date", "industry", f"stockPrice_{column_join_word}_lag{c}"]
        )
        for item, c in zip(lags, shifts)
    ]

    windows = [4, 8, 10, 14]
    aves = [df_input.shift(1).rolling(window=w).mean() for w in windows]
    aves = [
        wide_to_long(
            item, columns=["date", "industry", f"stockPrice_{column_join_word}_ave{c}"]
        )
        for item, c in zip(aves, windows)
    ]

    return lags + aves


stockPrice_by_industry_lagFetures = []
stockPrice_by_industry_lagFetures.append(
    create_lag_features(long_to_wide(stockPrice_by_industry_mean), "mean")
)
stockPrice_by_industry_lagFetures.append(
    create_lag_features(long_to_wide(stockPrice_by_industry_min), "min")
)
stockPrice_by_industry_lagFetures.append(
    create_lag_features(long_to_wide(stockPrice_by_industry_max), "max")
)
stockPrice_by_industry_lagFetures.append(
    create_lag_features(long_to_wide(stockPrice_by_industry_median), "median")
)
stockPrice_by_industry_lagFetures = [
    item for l in stockPrice_by_industry_lagFetures for item in l
]

In [70]:
print(data_long_merged_drop.shape)
data = pd.merge(
    data_long_merged_drop, industry_list, left_on="symbol", right_on="Symbol", how='left'
).drop("Symbol", axis=1)
data = data.rename(columns={'Industry': 'industry'})
print(data.shape)

for feature_df in stockPrice_by_industry_lagFetures:
    data = pd.merge(
        data,
        feature_df,
        on=["date", "industry"],
        how="left",
    )

labelEncoder_industry = LabelEncoder()
data.loc[:, 'industry'] = labelEncoder_industry.fit_transform(data.loc[:, 'industry'])
print(data.shape)

(1330868, 11)
(1330868, 12)
(1330868, 44)


In [71]:
# 学習データ, テストデータ(コンペでの予測対象のデータ)に分割
splitDate = dt.datetime.strptime("2019-11-24", "%Y-%m-%d")
test = data[data["date"] >= splitDate]
train = data[data["date"] < splitDate]

# 学習データ, validationデータに分割. 学習データのうち、最新1ヶ月をvalidationデータにする
splitDate = max(train.iloc[:, 0]) - dt.timedelta(days=28)
val = train[train["date"] > splitDate]
train = train[train["date"] <= splitDate]

In [72]:
train_x = train.drop(["date", "stockPrice"], axis=1)
train_y = train["stockPrice"]
val_x = val.drop(["date", "stockPrice"], axis=1)
val_y = val["stockPrice"]
test_x = test.drop(["date", "stockPrice"], axis=1)
test_y = test["stockPrice"]

In [73]:
import xgboost as xgb

In [74]:
train_data = xgb.DMatrix(train_x, label=train_y)
eval_data = xgb.DMatrix(val_x, label=val_y)
test_data = xgb.DMatrix(test_x, label=test_y)

xgb_params = {"objective": "reg:squarederror", "eval_metric": "mae"}
evals = [(train_data, "train"), (eval_data, "eval")]

gbm = xgb.train(
    xgb_params,
    train_data,
    num_boost_round=100,
    early_stopping_rounds=10,
    evals=evals,
)

[0]	train-mae:21.64202	eval-mae:29.19699
[1]	train-mae:15.15658	eval-mae:20.50580
[2]	train-mae:10.61579	eval-mae:14.42672
[3]	train-mae:7.43842	eval-mae:10.16879
[4]	train-mae:5.22200	eval-mae:7.19898
[5]	train-mae:3.68552	eval-mae:5.13747
[6]	train-mae:2.63495	eval-mae:3.71524
[7]	train-mae:1.93796	eval-mae:2.74542
[8]	train-mae:1.49621	eval-mae:2.10982
[9]	train-mae:1.23099	eval-mae:1.70994
[10]	train-mae:1.07960	eval-mae:1.46784
[11]	train-mae:0.99645	eval-mae:1.32621
[12]	train-mae:0.95201	eval-mae:1.24651
[13]	train-mae:0.92855	eval-mae:1.19938
[14]	train-mae:0.91623	eval-mae:1.17101
[15]	train-mae:0.90950	eval-mae:1.15581
[16]	train-mae:0.90550	eval-mae:1.14541
[17]	train-mae:0.90343	eval-mae:1.13966
[18]	train-mae:0.90220	eval-mae:1.13729
[19]	train-mae:0.90138	eval-mae:1.13546
[20]	train-mae:0.90069	eval-mae:1.13462
[21]	train-mae:0.90054	eval-mae:1.13414
[22]	train-mae:0.90007	eval-mae:1.13348
[23]	train-mae:0.89975	eval-mae:1.13336
[24]	train-mae:0.89958	eval-mae:1.13312
[25

In [79]:
predict_y = gbm.predict(eval_data)
predict_y.shape, val_y.shape

((13112,), (13112,))

In [80]:
from sklearn.metrics import mean_squared_error

mean_squared_error(predict_y, val_y)

7.12583213183696

In [81]:
predict_result = gbm.predict(test_data)
result = pd.DataFrame(
    [labelEncoder.inverse_transform(test["symbol"]), predict_result]
).T
result.columns = ["id", "y"]
result.head()

,id,y
0,VGSH,60.395943
1,JEF,20.484234
2,IVZ,17.016321
3,KTCC,5.578807
4,FBZ,14.557187


In [82]:
result["y"].min(), result.shape

(0.2678117, (3278, 2))

In [83]:
result.to_csv("../data/predict/nb_005_xgboost.csv", index=False)